In [2]:
import numpy as np 

In [56]:
y = np.array([[0.],
       [1.],
       [1.]])


MAX_ITERS = 2
GAMMA = 0.1


tx = np.array([[2.3, 3.2],
       [1. , 0.1],
       [1.4, 2.3]])

initial_w = np.array([[0.409111], [0.843996]])

In [57]:
np.sqrt(np.sum(initial_w**2)), 

(0.9379238019887329,)

In [93]:

def compute_logistic_loss(y, tx, w, lambda_): 

    pred = np.dot(tx, w)
    sigmoids = 1.0 / (1 + np.exp(-pred))
    loss = -0.5 * np.mean(y * np.log(sigmoids) + (1 - y) * np.log(1 - sigmoids)) + (lambda_ ) * np.linalg.norm(initial_w) ** 2
    return sigmoids, loss

def reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma):
    """
    Perform regularized logistic regression using gradient descent.
    
    Parameters:
    y: np.array
        The target values
    tx: np.array
        The data matrix (each row is a data point)
    initial_w: np.array
        Initial weights
    max_iters: int
        Maximum number of iterations for gradient descent
    gamma: float
        Learning rate
    lambda_: float
        Regularization strength for L2 penalty

    Returns:
    w: np.array
        Optimized weights after training
    """
    
    w = initial_w
    
    sigmoids, loss = compute_logistic_loss(y, tx, w, lambda_)

    
    for iter in range(max_iters):

        # compute the gradient
        grad = tx.T.dot(sigmoids - y)/len(y) + 2 * lambda_ * w

        # update w through the negative gradient direction
        w = w - gamma * grad

        sigmoids, loss = compute_logistic_loss(y, tx, w, lambda_)
        
    return w, loss

reg_logistic_regression(y, tx, 1.0, initial_w,  0, GAMMA)

(array([[0.409111],
        [0.843996]]),
 1.5833647530305026)

In [95]:

def compute_logistic_loss(y, tx, w): 

    pred = np.dot(tx, w)
    sigmoids = 1.0 / (1 + np.exp(-pred))
    loss = -np.mean(y * np.log(sigmoids) + (1 - y) * np.log(1 - sigmoids))
    return sigmoids, loss



def logistic_regression(y, tx, initial_w, max_iters, gamma):
    """
    Perform logistic regression using gradient descent.
    
    Parameters:
    y: np.array
        The target values
    tx: np.array
        The data matrix (each row is a data point)
    initial_w: np.array
        Initial weights
    max_iters: int
        Maximum number of iterations for gradient descent
    gamma: float
        Learning rate

    Returns:
    w: np.array
        Optimized weights after training
    """
    
    w = initial_w

    sigmoids, loss = compute_logistic_loss(y, tx, w)
    
    for iter in range(max_iters):
        # compute the gradient

        grad =  tx.T.dot(sigmoids - y)/len(tx)

        # update w through the negative gradient direction
        w = w - gamma * grad

        sigmoids, loss = compute_logistic_loss(y, tx, w)
        
    return w, loss

logistic_regression(y, tx, initial_w,  0, GAMMA)

(array([[0.409111],
        [0.843996]]),
 1.4073273893870055)